In [1]:
import numpy as np
import math
import pandas as pd
import random

In [5]:
def make_tour_for_testing(grid, n_chargers, length):
    
    tour = [0]
    dist = 0
    nodes = list(range(1+n_chargers,len(grid)))
    random.shuffle(nodes)
    curr_node = 0
    i = 1
    while i<=length:
        new = nodes.pop()
        tour.append(new)
        x1, y1 = grid[curr_node]
        x2, y2 = grid[new]
        dist += distance(x1,y1,x2,y2)
        curr_node = new
        i+=1
    
    tour.append(0)
    x1, y1 = grid[curr_node]
    x2, y2 = grid[0]
    dist += distance(x1,y1,x2,y2)
      
    return tour, dist

def shortest_detour(grid, p1,p2, points): #For the CS-matrix where i!=j
    closest_dist = math.inf
    closest_cs = None
    dist_to_cs = 0
    
    for point in points:
        detourlength = detour_length(p1,p2,point,grid)
        
        if detourlength<closest_dist:
           closest_cs = point
           dist_to_cs = grid_distance(p1,point,grid)
           closest_dist=detourlength
    
    return closest_cs, dist_to_cs

def closest_cs(grid,p1,points): #For the CS-matrix where i==j
    closest_dist = math.inf
    closest_cs = None
    
    for point in points:
        
        dist = grid_distance(p1,point,grid)
        
        if dist<closest_dist:
            closest_dist = dist
            closest_cs = point
            
    return closest_cs, closest_dist

def distance(x1,y1,x2,y2):
    
    return math.sqrt((x2-x1)**2+(y2-y1)**2)

def grid_distance(p1,p2,grid):
    x1,y1 = grid[p1]
    x2,y2 = grid[p2]
    
    return distance(x1,y1,x2,y2)
    

def detour(p1,new,trip):
    
    new_trip = trip.copy()
    index = new_trip.index(p1) + 1
    new_trip.insert(index,(new))
    
    return new_trip,index

def detour_length_pp(p1,p2,cs,preproseccing):
    
    distances = preproseccing["Distances"]
    
    
    new_length = distances[p1,cs]+distances[cs,p2]
    old_length = distances[p1,p2]
    
    return new_length-old_length

def detour_length(p1,p2,point,grid):
    
    new_length = grid_distance(p1,point,grid)+grid_distance(point,p2,grid)
    old_length = grid_distance(p1,p2,grid)
    
    return new_length-old_length


def get_cum_range(preproseccing,trip,battery_range):
    _, cum_length = tourlength(trip, preproseccing)
    
    return [battery_range-x for x in cum_length]

def tourlength(tour,preproseccing):
    
    distances = preproseccing["Distances"]
    tourlength = 0
    cum_length = [0]

    for i in range(len(tour)-1):
        p1 = tour[i]
        p2 = tour[i+1]
        tourlength += distances[p1,p2]
        cum_length.append(tourlength)
    
    return tourlength, cum_length

def new_tour(tour,p1,p2,cs,cum_range,preproseccing):
    
    distances = preproseccing["Distances"]
    
    node1, node2 = p1[0], p2[0]
    
    de_tour, index = detour(p1,(cs,0),tour) # Make a detour by adding cs at index after p1
    
    increase_in_len = detour_length_pp(node1,node2,cs,preproseccing) #How much the tourlength is increased by taking the detour
    
    range_cs = cum_range[index-1] - distances[node1,cs] #Calculating the cumulative range at the charging station
    
    cum_range[index:] = [x-increase_in_len for x in cum_range[index:]] #subtracting the increase in length for each node after the cs
    
    cum_range.insert(index,range_cs) #Insert the cumulative range at the cs into the list for the trip
    
    return de_tour, cum_range


In [3]:
def preffered_cs(problem): #Charge the parameters so that the n_chargers and n_jobs are retrieved from the problem
    '''
    This function should take all the nodes and order them in a nxn matrix where n is the number of offshore wind turbines. In each position in the matrix
    there should be placed a tuple (closest charger, minimum charge needed to go here from start node (column-node))
    
    output: Matrix reads as i = row, j = col
    
    PROBLEMS:
    
    Should I include the depot in the charging matrix? This could maybe be added at the end.
    '''
    nodes = problem["Locations"]
    
    n_chargers = len(problem["ChargingStations"])
    n_nodes = len(nodes)
    matrix = np.empty((n_nodes,n_nodes),dtype=object)
    
    charging_stations = list(range(1,n_chargers+1))
    
    
    for i in range(n_nodes):
        
        
        for j in range(n_nodes):
            
            if i<n_chargers+1:
                cs, dist_to_cs = i,0
                matrix[i,j] = (cs,round(dist_to_cs,2))
        
            elif i == j:
                #Add the nearest charging station to i
                cs, dist_to_cs = closest_cs(nodes, i,charging_stations)
                matrix[i,j] = (cs,round(dist_to_cs,2))
                
            else: #Add the chargin station with the shortest detour
                cs, dist_to_cs = shortest_detour(nodes,i,j,charging_stations)
                matrix[i,j] = (cs,round(dist_to_cs,2))
                
    return matrix

In [4]:
def all_distances(problem):
    '''This fnction should take all the nodes and make a nxn matrix where n is the total number of nodes in the problem.
    Each element in the matrix will be the distance from i to j.
    
    The purpose of this is to always have the distances between nodes stored.
    
    @params problem
    @params cs_matrix: Matrix contatining information about the closest cs when going from i to j
    
    @output matrix with disances between all nodes
    
    output: Matrix reads as i = row, j = col'''
    nodes = problem["Locations"]
    n_nodes = len(nodes)
    matrix = np.empty((n_nodes,n_nodes))

    
    
    for i in range(n_nodes):
        
        for j in range(n_nodes):
            dist = round(grid_distance(i,j,nodes),2)
            
            matrix[i,j] = dist
            
    
    return matrix

In [ ]:
def all_distances_with_charging(problem,cs_matrix):
    '''This fnction should take all the nodes and make a nxn matrix where n is the total number of nodes in the problem.
    Each element in the matrix will be the distance from i to j + the distance from j to the nearest charging station (it is only relevant to add the distance to cs for the offshore wind turbines).
    
    The purpose of this is to be able to say when we are at a given node if we can, with the current battery range, go to the next node and beyond.
    
    @params problem
    @params cs_matrix: Matrix contatining information about the closest cs when going from i to j
    
    @output matrix with minimum battery range to go on
    
    output: Matrix reads as i = row, j = col'''
    nodes = problem["Locations"]
    n_nodes = len(nodes)
    matrix = np.empty((n_nodes,n_nodes))
    
    
    distance_to_cs = [cs_matrix[i, i][1] for i in range(len(cs_matrix))]
    
    
    #Loop through all nodes and find the distance to the closest cs
    
    
    for i in range(n_nodes):
        
        for j in range(n_nodes):
            dist = round(grid_distance(i,j,nodes),2)
            
            cs_dist = distance_to_cs[j]
            
            matrix[i,j] = dist+cs_dist
            
    
    return matrix

In [1]:
def preprocessing(problem):
    """Function to easily retrive the matrices made before starting to solve to problem. Both of these are used when adding charging in postproseccing.

    Args:
        problem dict: Dictionary holding the tags and coordinates of all the nodes
        int: number of chargers in the problem

    Returns:
        tuple: returns a tuple of the preferred cs matrix and the distance matrix
    """
    pp = {}
    
    
    
    cs_matrix = preffered_cs(problem)
    d_matrix = all_distances(problem)
    dwc_matrix = all_distances_with_charging(problem,cs_matrix)
    
    pp["PreferredCS"] = cs_matrix
    pp["Distances"] = d_matrix
    pp["DistancesWithCharging"] = dwc_matrix
    
    return pp